In [257]:
import pandas as pd
import re
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [258]:
df = pd.read_csv('salespeople.csv')

In [259]:
X = df.drop('promoted', axis=1)
y = df.promoted

In [260]:
df.drop(350, axis=0, inplace=True)

In [261]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.1, random_state=1234)

In [262]:
numeric_pipeline = Pipeline([
    ('scaler', StandardScaler())
])

In [263]:
categorical_pipeline = Pipeline([
    ('encoder', OneHotEncoder())
])

In [265]:
class PrepProcesor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.salesImputer = SimpleImputer()

    def fit(self, X, y=None):
        self.salesImputer.fit(X[['sales']])
        return self

    def transform(self, X, y=None):
        X['sales'] = self.salesImputer.transform(X[['sales']])
        return X

    def predict(self, X):
        return self.model.predict(X)

transformer = ColumnTransformer([('num', numeric_pipeline, ['sales', 'customer_rate', 'performance']), ('cat', categorical_pipeline, [])])


In [267]:
# Select only the non-numeric columns
non_numeric_columns = df.select_dtypes(exclude=[np.number])

# Print the column names
print(non_numeric_columns.columns)

Index([], dtype='object')


In [256]:
!pip install sklearn xgboost

In [268]:
# Define the machine learning pipeline
mlpipe = Pipeline([
    ('InitialPreproc', PrepProcesor()),
    ('Transformer', transformer),
    ('xgb', XGBClassifier())
])

# Fit the pipeline to the training data
mlpipe.fit(X_train, y_train)

Pipeline(steps=[('InitialPreproc', PrepProcesor()),
                ('Transformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['sales', 'customer_rate',
                                                   'performance']),
                                                 ('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder())]),
                                                  [])])),
                ('xgb',
                 XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
                               colsample_bylevel=1, colsample_b...
                               feature_types=None, gamma=0, gpu_id=-1,
                          

In [269]:
# Define the model variable
model = mlpipe

In [270]:
yhat = model.predict(X_test)

In [271]:
precision_score(y_test, yhat)


1.0

In [272]:
recall_score(y_test, yhat)


0.9230769230769231

In [273]:
import joblib


In [274]:
joblib.dump(model, 'xgbpipe.joblib')

['xgbpipe.joblib']